In [1]:
import sys
sys.path.append('./nglod/sdf-net')
import argparse

In [2]:
from lib.trainer import Trainer
from train_nglod import build_parser, args_to_str, CustomizedTrainer

/titan/bohdan/miniconda3/envs/nglod/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
parser = build_parser()
args = parser.parse_args([
    '--net', 'OctreeSDF', 
    '--num-lods', '5', 
    '--dataset-path', '../../test_task_meshes/0.obj', 
    '--epoch', '250',
    '--exp-name', '0'
    ])
args_str = args_to_str(args, parser)

In [ ]:
model = CustomizedTrainer(args, args_str)
# model.train()